In [1]:
import numpy as np

out = np.load('training-data.npz')
th_train = out['th'] #th[0],th[1],th[2],th[3],...
u_train = out['u'] #u[0],u[1],u[2],u[3],...

data = np.load('test-prediction-submission-file.npz')
upast_test = data['upast'] #N by u[k-15],u[k-14],...,u[k-1]
thpast_test = data['thpast'] #N by y[k-15],y[k-14],...,y[k-1]
# thpred = data['thnow'] #all zeros


def create_IO_data(u,y,na,nb):
    X = []
    Y = []
    for k in range(max(na,nb), len(y)):
        X.append(np.concatenate([u[k-nb:k],y[k-na:k]]))
        Y.append(y[k])
    return np.array(X), np.array(Y)

na = 8
nb = 3
Xtrain, Ytrain = create_IO_data(u_train, th_train, na, nb)


import pickle
# Load the saved model and kernel using pickle
with open('GPmodel.dump', 'rb') as file:
    reg = pickle.load(file)

# Print the kernel configuration
print(reg.kernel_)


Ytrain_pred = reg.predict(Xtrain)
print('train prediction errors:')
print('RMS:', np.mean((Ytrain_pred-Ytrain)**2)**0.5,'radians')
print('RMS:', np.mean((Ytrain_pred-Ytrain)**2)**0.5/(2*np.pi)*360,'degrees')
print('NRMS:', np.mean((Ytrain_pred-Ytrain)**2)**0.5/Ytrain.std()*100,'%')

 #only select the ones that are used in the example
Xtest = np.concatenate([upast_test[:,15-nb:], thpast_test[:,15-na:]],axis=1)

Ypredict = reg.predict(Xtest)
assert len(Ypredict)==len(upast_test), 'number of samples changed!!'

np.savez('NARX-GP-prediction-submission-file.npz', upast=upast_test, thpast=thpast_test, thnow=Ypredict)

RBF(length_scale=4.31) + WhiteKernel(noise_level=9.56e-06)
train prediction errors:
RMS: 0.004013156526155631 radians
RMS: 0.22993693147410038 degrees
NRMS: 0.818188634684261 %
